In [ ]:
from bs4 import BeautifulSoup as bs
import requests
from konlpy.tag import Okt
import speech_recognition as sr
import glob
from pydub import AudioSegment

# 질의응답 함수
def qna_bot(text):
  word_dic = {}  
  okt = Okt()
  sum_text = ''

  tokenized_doc = okt.pos(text)
  
  for word in tokenized_doc:

    if word[1] == 'Noun':
    
      url = 'https://search.naver.com/search.naver?sm=tab_hty.top&where=kdic&query={}'.format(word[0])
      html = requests.get(url)
      soup = bs(html.text, 'html.parser')
      try:
        temp_sum = soup.select('span[class="lnk_sub_tit elss"]')[0].text
      except:
        word_dic[word[0]] = ['키워드 검색결과 없음', url]
        continue
      
      lnk = soup.select('a[class="lnk_tit"]')[0].attrs['href']

      if temp_sum != sum_text:
        sum_text = temp_sum
        word_dic[word[0]] = [sum_text, lnk]
      else:
        word_dic[word[0]] = ['링크에서 내용을 확인해주세요', lnk]
  
  return word_dic


def voice():
    try:
        r = sr.Recognizer()
        
        # with sr.Microphone() as source:
        #     print('음성을 입력하세요.')
        #     audio = r.listen(source)


        # with open('C:/DEV/STUDY/FinalProject/flasktest/static/wav/qna.wav', "wb") as file:   # open in binary mode
        #   response = requests.get(url)
        #   file.write(response.content)
        
        output = glob.glob('C:/Users/admin/Downloads/*.weba')
        
        file = output[-1]
        
        audioSegment = AudioSegment.from_file(file, 'webm')
        
        new_file_path = 'C:/Users/admin/Downloads/qna.wav'
        
        audioSegment.export(new_file_path, format='wav')
        
        row_source = sr.AudioFile(new_file_path)

        with row_source as source:
            audio = r.record(source)
            try:
                print('음성변환 : ' + r.recognize_google(audio, language='ko-KR'))
                text_dict = qna_bot(r.recognize_google(audio, language='ko-KR'))
                row_list = []
                for key, val in text_dict.items():
                    temp_dict = {'key' : key, 'txt' : val[0], 'lnk' : val[1]}
                    row_list.append(temp_dict)

            except :
                pass

    except KeyboardInterrupt:
        pass

    return row_list